# DataFrame <-> MongoDB

這個 .ipynb 示範怎麼將 DataFrame 存入及讀出 MongoDB。

In [ ]:
% matplotlib inline

import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import json
from pymongo import MongoClient

In [ ]:
# 讀取指定日期之後的股價資訊
df=web.DataReader("TSLA", 'yahoo', datetime(2016,1,1))

## 將 DataFrame 存入 MongoDB

In [ ]:
df['symbol'] = 'TSLA'
df['date'] = df.index

In [ ]:
df.T.to_json()

In [ ]:
conn = MongoClient()

db = conn.stocks

In [ ]:
records = json.loads(df.T.to_json()).values()
db.prices.insert_many(records)

## 從 MongoDB 讀出 DataFrame

In [ ]:
cursor = db.prices.find({'symbol':'TSLA'})

In [ ]:
df1 = pd.DataFrame(list(cursor))

In [ ]:
# 關於日期的轉換
# 參考：http://stackoverflow.com/questions/25455067/pandas-dataframe-datetime-index-doesnt-survive-json-conversion-and-reconversion
df1['date'] = pd.to_datetime(df1['date'], unit='ms')

In [ ]:
df1.sort_values('date')

In [ ]:
#db.prices.delete_many({'symbol':'TSLA'})

In [ ]:
df1 = df1.set_index('date')

In [ ]:
del df1['_id']

In [ ]:
df1.sort_index()

In [ ]:
df1[['Close', 'Volume']].plot(subplots=True)